In [22]:
%load_ext autoreload
%autoreload 2

from datasets import load_dataset
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import urllib.request
# urllib.request.urlretrieve("https://artifacts.opensearch.org/models/ml-models/amazon/gpt/GPT2_xl_sqg/1.0.0/generate.py", "generate.py")
import generate
from generate import Synthetic_Query_Generation
from opensearch_py_ml.ml_models import SentenceTransformerModel

# pd.set_option('display.expand_frame_repr', None)
# pd.set_option('max_colwidth', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def myselect(x):
    if max(x["passages"]["is_selected"]) == 1:
        return x["passages"]["passage_text"][np.argmax(x["passages"]["is_selected"])]
    return "-1"

In [2]:
ds = load_dataset("ms_marco", "v1.1")

In [3]:
df = pd.DataFrame.from_dict(ds['validation'])

In [13]:
df['passage'] = df.apply(lambda x: myselect(x), axis=1)

In [15]:
df = df[["query", "passage"]][df.passage != "-1"]
df

,query,passage
0,walgreens store sales average,The average Walgreens salary ranges from appro...
1,how much do bartenders make,"According to the Bureau of Labor Statistics, t..."
2,what is a furuncle boil,"A boil, also called a furuncle, is a deep foll..."
3,what can urinalysis detect,Urinalysis is a test that evaluates a sample o...
4,what is vitamin a used for,"Vitamin A is also used for shigellosis, diseas..."
...,...,...
10042,what is nmr and mri,Nuclear magnetic resonance (NMR) is a physical...
10043,where is sandon,Sandon is a ghost town in the Kootenay region ...
10044,where do you go to apply for a mortgage,Applying for the Mortgage: Information Banks W...
10045,part of brain the makes us,The biggest part of the brain is the cerebrum....


In [18]:
sample_passages = list(df.passage.values)
len(sample_passages)

9706

In [19]:
sample_passages[:8]

['The average Walgreens salary ranges from approximately $15,000 per year for Customer Service Associate / Cashier to $179,900 per year for District Manager. Average Walgreens hourly pay ranges from approximately $7.35 per hour for Laboratory Technician to $68.90 per hour for Pharmacy Manager. Salary information comes from 7,810 data points collected directly from employees, users, and jobs on Indeed.',
 'According to the Bureau of Labor Statistics, the average hourly wage for a bartender is $10.36, and the average yearly take-home is $21,550. Bartending can be a lot of things. For some it is exciting, for others exhausting. At times there is a lot of fun to be had, at others it is rather dull. But for the most part, bartending is almost always rewarding in the financial sense, as long as you stick with it.',
 'A boil, also called a furuncle, is a deep folliculitis, infection of the hair follicle. It is most commonly caused by infection by the bacterium Staphylococcus aureus, resulting

In [20]:
ss = Synthetic_Query_Generation(sentences=sample_passages[:8])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
step_query = ss.generate_synthetic_queries(num_gpu=1, tokenizer_max_length=300, total_queries=10, numseq=5, toxic_cutoff=0.01, tokens_to_word_ratio=0.6)

Tokenizing corpus... This might take a while...
Preparing input_ids and attention_mask... 


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 355.01it/s]

<class 'str'>
The average Walgreens salary ranges from approximately $15,000 per year for Customer Service Associate / Cashier to $179,900 per year for District Manager. Average Walgreens hourly pay ranges from approximately $7.35 per hour for Laboratory Technician to $68.90 per hour for Pharmacy Manager. Salary information comes from 7,810 data points collected directly from employees, users, and jobs on Indeed.
<class 'str'>
According to the Bureau of Labor Statistics, the average hourly wage for a bartender is $10.36, and the average yearly take-home is $21,550. Bartending can be a lot of things. For some it is exciting, for others exhausting. At times there is a lot of fun to be had, at others it is rather dull. But for the most part, bartending is almost always rewarding in the financial sense, as long as you stick with it.
<class 'str'>
A boil, also called a furuncle, is a deep folliculitis, infection of the hair follicle. It is most commonly caused by infection by the bacterium 

The number of steps for creating queries:  8
Running on CPU...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [02:53<00:00, 21.67s/it]
Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.3-alpha/toxic_debiased-c7548aa0.ckpt" to /home/elastic/.cache/torch/hub/checkpoints/toxic_debiased-c7548aa0.ckpt
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 476M/476M [00:04<00:00, 121MB/s]


The total number of synthetic queries before detoxify is 80
70 good queries are kept after detoxify.
File is saved to /home/elastic/notebooks/vector_search_applications/queries_after_detoxify/synthetic_queries_batch_.p file.
Zip file is saved to/home/elastic/notebooks/vector_search_applications/clean_synthetic_queries.zip


In [31]:
custom = SentenceTransformerModel(folder_path='/home/elastic/notebooks/vector_search_applications/models/', overwrite=True)

In [32]:
df = custom.read_queries(read_path='./clean_synthetic_queries.zip', overwrite=True)

Reading synthetic query file: /home/elastic/notebooks/vector_search_applications/models/synthetic_queries/synthetic_queries_batch.p



In [33]:
df

,prob,query,passages
39,0.167991,how does vitamins work,"Vitamin A is also used for shigellosis, disea..."
21,326.843933,what causes boils,"A boil, also called a furuncle, is a deep fol..."
18,30.521439,how much money do bartenders make,"According to the Bureau of Labor Statistics, ..."
9,10.467592,what is the starting wage at walgreen,The average Walgreens salary ranges from appr...
30,0.016274,what is urinary culture,Urinalysis is a test that evaluates a sample ...
...,...,...,...
44,38.914867,what are vitamins good for,"Vitamin A is also used for shigellosis, disea..."
43,0.160010,is vit d good for anemia,"Vitamin A is also used for shigellosis, disea..."
57,0.786869,what is average basement wall,"Our free calculator uses recent, trusted data..."
59,0.001369,how much does it cost per frame of wall,"Our free calculator uses recent, trusted data..."


In [28]:
import torch
import os
torch.cuda.empty_cache()

In [ ]:
os.cpu_count

In [34]:
training = custom.train(read_path = './clean_synthetic_queries.zip',
                        output_model_name = 'test_model.pt',
                        zip_file_name= 'test_model.zip',
                        overwrite = True,
                        num_epochs = 10,
                        verbose = True,
                        num_gpu=1,
                        num_machines=os.cpu_count())

Reading synthetic query file: /home/elastic/notebooks/vector_search_applications/models/synthetic_queries/synthetic_queries_batch.p

Loading training examples... 



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 66/66 [00:00<00:00, 699050.67it/s]


Start training without accelerator...

The number of training epoch are 10

The total number of steps per training epoch are 2

Training epoch 0...



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.64it/s]


Training epoch 1...



...

Training epoch 8...



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  7.23it/s]


Training epoch 9...



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  7.20it/s]


Total training time: 4.269590616226196

Preparing model to save...

Model saved to path: /home/elastic/notebooks/vector_search_applications/models/

tokenizer_json_path:  /home/elastic/notebooks/vector_search_applications/models/tokenizer.json
zip file is saved to /home/elastic/notebooks/vector_search_applications/models/test_model.zip



In [35]:
type(training)

NoneType

In [36]:
from opensearch_interface import OpenSearchClient

In [39]:
client = OpenSearchClient()

In [43]:
client.show_indexes()

health status index                        uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   semantic-538-testrun         DjBPg6CdQwKbOGhJrI4YIQ   3   1        284            0      2.9mb          2.9mb
yellow open   security-auditlog-2023.09.10 q2UCytHxQXOdDxdpwOeqpA   1   1       1704            0      1.5mb          1.5mb
yellow open   test-kw-index                6EF4Q2xDT9Gz1wua5a2IpQ   3   1        158            0      5.6mb          5.6mb
yellow open   security-auditlog-2023.09.11 u3bVAn9lRReojRlF0t3sYw   1   1       1704            0      3.3mb          3.3mb
yellow open   kw-full                      uNhdaqbnRVuyJci_L1Om8Q   3   1       6678            0     12.1mb         12.1mb
yellow open   security-auditlog-2023.09.12 wJoWS1RfSBynSDjh3tV7Mw   1   1       1704            0      3.2mb          3.2mb
yellow open   semantic-full                ydvbifM0Rcu0DBz-rjCHzQ   3   1       6678            0     74.2mb         74.2mb
green  o

In [42]:
import opensearch_py_ml as oml